In [ ]:
#!pip install transformers tensorflow datasets simpletransformers

In [ ]:
import json
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from transformers import AutoTokenizer
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
with open("/content/train.json", "r") as file:
    train = json.load(file)

In [ ]:
train

[{'context': 'Vin is a Mistborn of great power and skill. She has the ability to ingest metals and use their powers. Her abilities make her one of the most powerful Mistborn in existence.',
  'qas': [{'question': "What is Vin's speciality?",
    'id': '0',
    'answers': [{'text': 'Mistborn of great power and skill',
      'answer_start': 0}]},
   {'question': 'What can Vin ingest to use powers?',
    'id': '1',
    'answers': [{'text': 'metals', 'answer_start': 72}]}]},
 {'context': 'The sun rises in the east and sets in the west. This has been a natural phenomenon for centuries, providing us with day and night cycles.',
  'qas': [{'question': 'Where does the sun rise?',
    'id': '2',
    'answers': [{'text': 'east', 'answer_start': 16}]},
   {'question': 'Where does the sun set?',
    'id': '3',
    'answers': [{'text': 'west', 'answer_start': 39}]}]},
 {'context': 'Albert Einstein was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars

In [ ]:
with open(r"/content/test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
test

[{'context': 'The Amazon Rainforest, also known as the Amazon Jungle, is a moist broadleaf forest in the Amazon biome that covers most of the Amazon Basin of South America. It is the largest tropical rainforest in the world.',
  'qas': [{'question': 'What is the Amazon Rainforest also known as?',
    'id': '3',
    'answers': [{'text': 'Amazon Jungle', 'answer_start': 45}]},
   {'question': 'Where is the Amazon Rainforest located?',
    'id': '4',
    'answers': [{'text': 'Amazon Basin of South America',
      'answer_start': 98}]}]},
 {'context': "Shakespeare's plays have been translated into every major language. His work includes tragedies like 'Hamlet', 'Macbeth', and 'Romeo and Juliet', comedies like 'A Midsummer Night's Dream', and histories like 'Richard III'.",
  'qas': [{'question': "What are some of Shakespeare's tragedies?",
    'id': '5',
    'answers': [{'text': "'Hamlet', 'Macbeth', and 'Romeo and Juliet'",
      'answer_start': 86}]},
   {'question': 'Which play by Shake

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
model_type="bert"
model_name= "bert-base-cased"
# if model_type == "bert":
#     model_name = "bert-base-cased"

# elif model_type == "roberta":
#     model_name = "roberta-base"

# elif model_type == "distilbert":
#     model_name = "distilbert-base-cased"

# elif model_type == "distilroberta":
#     model_type = "roberta"
#     model_name = "distilroberta-base"

# elif model_type == "electra-base":
#     model_type = "electra"
#     model_name = "google/electra-base-discriminator"

# elif model_type == "electra-small":
#     model_type = "electra"
#     model_name = "google/electra-small-discriminator"

# elif model_type == "xlnet":
#     model_name = "xlnet-base-cased"

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 3
model_args.num_train_epochs = 5
model_args.max_seq_length = 128
model_args.use_cuda = False

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "use_wandb": False
}

In [ ]:
model = QuestionAnsweringModel(model_type, model_name, args=train_args)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(train, eval_data=test)

Could not find answer: 'Vin is a Mistborn of great power' vs. 'Mistborn of great power and skill'
convert squad examples to features:   0%|          | 0/14 [00:00<?, ?it/s]Could not find answer: 'in the' vs. 'east'
Could not find answer: 'work is also known for its influence on the' vs. 'influence on the philosophy of science'
Could not find answer: 'Mars in Paris,' vs. 'Paris, France'
Could not find answer: 'is the largest rainforest in the' vs. 'largest rainforest in the world'
Could not find answer: 'crucial role in regulating the' vs. 'regulating the Earth's climate'
Could not find answer: 'playwright, and his influence on literature' vs. 'influence on literature and culture'
add example index and unique id: 100%|██████████| 14/14 [00:00<00:00, 112706.83it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(1,
 {'global_step': [1],
  'correct': [0],
  'similar': [2],
  'incorrect': [2],
  'train_loss': [4.852957725524902],
  'eval_loss': [0.0175933837890625]})

In [ ]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Make predictions with the model
to_predict = [{
  "context": "Vin is a Mistborn of great power and skill. She has the ability to ingest metals and use their powers. Her abilities make her one of the most powerful Mistborn in existence.",
  "qas": [
    {
      "question": "What is Vin's speciality?",
      "id": "0",
      "answers": [
        {
          "text": "Mistborn of great power and skill",
          "answer_start": 0
        }
      ]
    },
    {
      "question": "What can Vin ingest to use powers?",
      "id": "1",
      "answers": [
        {
          "text": "metals",
          "answer_start": 72
        }
      ]
    }
  ]
}
]

In [ ]:
answers, probabilities = model.predict(to_predict)
for answer, prob in zip(answers, probabilities):
    print(f"Answer: {answer}")
    print(f"Probability: {prob}")

add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 19691.57it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: {'id': '0', 'answer': ['empty']}
Probability: {'id': '0', 'probability': [0.6191834862181853]}
Answer: {'id': '1', 'answer': ['empty']}
Probability: {'id': '1', 'probability': [0.6237207254690718]}
